In [5]:
import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [ ]:
# ------------- * FUNCTIONS * -------------

import os
import pickle

def get_content(file_path):
    data = pd.read_csv(file_path)
    labels = data['Health_Issue']
    features = data.drop('Health_Issue', axis=1)
    
    # ..transforming categorical values into numeric values..
    for column in features.columns:
        if(features[column].dtype == 'object'):
            features[column] = features[column].astype('category').cat.codes
            
    testing_samples = int(20 * len(data) / 100)
    
    testing_labels = labels.iloc[:testing_samples].to_numpy()
    testing_features = features.iloc[:testing_samples].to_numpy()
    
    training_labels = labels.iloc[testing_samples:].to_numpy()
    training_features = features.iloc[testing_samples:].to_numpy()
    
    scaler = StandardScaler()
    # ..the scaler is used to normalize input data so
    # it all fit into the same scale, because, it there's numbers
    # that are too distinct, the bigger ones have more weight 
    # in the models result, and it leads to bigger errors..
            
    return testing_labels, scaler.fit_transform(testing_features), training_labels, scaler.fit_transform(training_features)

def initialize_weights(input_dim, output_dim):
    stddev = np.sqrt(2.0 / input_dim)
    weights = np.random.normal(loc=0.0, scale=stddev, size=(input_dim, output_dim))
    
    return weights
    # ..the weights being too big or too small can
    # impact directly the learning process, in this case,
    # he initialization is applied because is a good
    # match for the relu activation function..

def test_model():
    sample = random.randrange(0, testing_samples)
    
    input_biased = np.hstack((bias, testing_features[sample]))
        
    output1 = np.tanh(weights1.dot(input_biased))
    output1_biased = np.hstack((bias, output1))
    
    output2 = np.tanh(weights2.dot(output1_biased))
    output2_biased = np.hstack((bias, output2))
    
    output3 = np.tanh(weights3.dot(output2_biased))
    output3_biased = np.hstack((bias, output3))
    
    result = np.tanh(weights4.dot(output3_biased))
    
    return result, testing_labels[sample]

def save_weights(weights, filename):
    np.save(filename, weights) 
        
def load_weights(filename):
    if not os.path.exists(filename):
        raise FileNotFoundError(f"No weights file found at {filename}")
    
    weights = np.load(filename, allow_pickle=True)  
    return weights

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

In [7]:
# ------------- * CONSTANTS * -------------

dataset = 'synthetic_covid_impact_on_work.csv'
testing_labels, testing_features, training_labels, training_features = get_content(dataset)

training_samples = len(training_features)
testing_samples = len(testing_features)

epocs = 100000 
# ..each epoc represents the time when all
# the data has been ran throught, if it's too big
# it's probably going to lead your model to overfitting..

learning_rate = 0.01
# ..keep it low, this value has 
# a lot of power in progressing the weights..

patterns = training_features.shape[1]
# ..how many features there is 
# to train from..

bias = 1
# ..changes the function's angle..

input_neurons = patterns
hidden_neurons1 = 82
hidden_neurons2 = 128
hidden_neurons3 = 64
output_neurons = 1

In [8]:

# ------------- * VARIABLES * -------------
weights1 = initialize_weights(input_neurons + 1, hidden_neurons1)
weights2 = initialize_weights(hidden_neurons1 + 1, hidden_neurons2)
weights3 = initialize_weights(hidden_neurons2 + 1, hidden_neurons3)
weights4 = initialize_weights(hidden_neurons3 + 1, output_neurons)
# ..the weights matrix need to have 1 column more
# because the bias is going to be inserted later on..

# weights1 -= 0.5
# weights2 -= 0.5
# weights3 -= 0.5
# weights4 -= 0.5
# # ..in this specific dataset, as the values are considerably 
# # small, if the weights are too high, it's going to lead
# # problems in the gradient calculation..

errors = np.zeros(training_samples)
errors_mean = np.zeros(epocs)

In [9]:
# ------------- * TRAINING * -------------
for i in range(epocs):
    for j in range(training_samples):
        
        # Forward Pass with ReLU activation
        input_biased = np.hstack((bias, training_features[j]))
        
        output1 = relu(weights1.dot(input_biased))
        output1_biased = np.hstack((bias, output1))
        
        output2 = relu(weights2.dot(output1_biased))
        output2_biased = np.hstack((bias, output2))
        
        output3 = relu(weights3.dot(output2_biased))
        output3_biased = np.hstack((bias, output3))
        
        result = relu(weights4.dot(output3_biased))
        
        # Error Calculation
        error = training_labels[j] - result
        errors[j] = (error ** 2) / 2
        
        # Backward Pass with ReLU derivative
        delta4 = error * relu_derivative(result)
        
        vdelta3 = weights4.transpose().dot(delta4)
        delta3 = vdelta3 * relu_derivative(output3_biased)
        
        vdelta2 = weights3.transpose().dot(delta3[1:]) # Skipping bias
        delta2 = vdelta2 * relu_derivative(output2_biased)
        
        vdelta1 = weights2.transpose().dot(delta2[1:]) # Skipping bias
        delta1 = vdelta1 * relu_derivative(output1_biased)
        
        # Update Weights
        
        weights1 += learning_rate * np.outer(delta1[1:], input_biased)
        weights2 += learning_rate * np.outer(delta2[1:], output1_biased)
        weights3 += learning_rate * np.outer(delta3[1:], output2_biased)
        weights4 += learning_rate * np.outer(delta4, output3_biased)
        
    errors_mean[i] = errors.mean()
    print(f"mean error of epoch: {errors_mean[i]}")
    
       # ..using samples the model didn't use for 
    # training to check it's performance..
    testing_times = 100
    testing_errors = np.zeros(testing_times)
    for k in range(testing_times):
        test_result, test_label = test_model()
        error = test_label - test_result
        testing_errors[k] = (error * error)/2
        
    # ..saving the weights to use it
    # later on..
    weights_dict = {
        'weights1': weights1,
        'weights2': weights2,
        'weights3': weights3,
        'weights4': weights4
    }
    save_weights(weights_dict, "weights_tanh")
            
    # ..stopping when reaching such accuracy to
    # avoid overfitting..
    testing_errors_mean = testing_errors.mean()
    if(testing_errors_mean < 0.08):
        print("early stoppin: model hit 85% or more of accuracy in testing five times.")
        break
    

C:\Users\Yasmim da Cunha\AppData\Local\Temp\ipykernel_47068\3243980398.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  errors[j] = (error ** 2) / 2


mean error of epoch: 0.10923257938964791


C:\Users\Yasmim da Cunha\AppData\Local\Temp\ipykernel_47068\3243980398.py:52: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  testing_errors[k] = (error * error)/2


mean error of epoch: 0.10582093250589061
mean error of epoch: 0.10494626109235894
mean error of epoch: 0.10417191445080959
mean error of epoch: 0.10340313256638234
mean error of epoch: 0.1025115793093382
mean error of epoch: 0.10161748062397025
mean error of epoch: 0.1007910418189057
mean error of epoch: 0.09973128661968647
mean error of epoch: 0.09867674924755787
mean error of epoch: 0.09749114916157868
mean error of epoch: 0.0963685684641414
mean error of epoch: 0.09535763582423785
mean error of epoch: 0.09407255453924278
mean error of epoch: 0.09287333754080002
mean error of epoch: 0.09145809542847844
mean error of epoch: 0.09027734044220494
mean error of epoch: 0.08905706354599324
mean error of epoch: 0.0876936098596524
mean error of epoch: 0.08616061400353665
mean error of epoch: 0.08478120461622182
mean error of epoch: 0.08348697616781306
mean error of epoch: 0.08205625526178352
mean error of epoch: 0.08036079080771347
mean error of epoch: 0.078657148514395
mean error of epoch: 0

KeyboardInterrupt: 

In [18]:
print(test_model())

(array([0.97247023]), np.int64(1))
